<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({
    'x1': [0, 0, 1, 0, 1, 1, 0],
    'x2': [0, 1, 0, 1, 0, 1, 0],
    'x3': [1, 1, 1, 0, 0, 1, 0],
    'y': [0, 1, 1, 1, 1, 0, 0,]
})

df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [13]:
q,w = check_X_y(df[['x1', 'x2', 'x3']], df['y'])
q.shape, w.reshape(-1, 1).shape

((7, 3), (7, 1))

In [11]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

In [30]:
class Perceptron(BaseEstimator, ClassifierMixin):
    def __init__(self, epochs=1000):
        self.epochs = epochs
        
        # define architecture of neural net
        self.inputs = 3
        self.hidden_nodes = 4
        self.outputs = 1
        
        # initialize random weights
        # input -> hidden weights
        self.weights_i2h = np.random.randn(self.inputs, self.hidden_nodes)
        # hidden -> output weights
        self.weights_h2o = np.random.randn(self.hidden_nodes, self.outputs)
    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_prime(self, s):
        sx = self.sigmoid(s)
        return sx / (1 - sx)
    
    def feed_forward(self, X):
        """Calculate the NN inference using feed forward"""
        # Weighted sum
        self.hidden_sum = np.dot(X, self.weights_i2h)
        
        # Activation
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum 2
        self.output_sum = np.dot(self.activated_hidden, self.weights_h2o)
        
        # Final Output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def back_prop(self, X, y, o):
        """ back prop through the network"""
        self.o_error = y - o
        
        # apply derivative of sigmoid to error
        self.o_delta = self.o_error * self.sigmoid_prime(self.output_sum)
        
        # z2 error: how much were our output layer weights off
        self.z2_error = self.o_delta.dot(self.weights_h2o.T)
        
        # z2 delta: how much were the weights off
        self.z2_delta = self.z2_error * self.sigmoid_prime(self.hidden_sum)
        
        # batching goes here
        # calculate partial gradient
        
        self.weights_i2h += X.T.dot(self.z2_delta) # adjust first set (input => hidden) weights
        self.weights_h2o += self.activated_hidden.T.dot(self.o_delta) # adjust second set (hidden => output) weights

    def train(self, X, y):
        o = self.feed_forward(X)
        self.back_prop(X, y, o)

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        y = y.reshape(-1, 1)
        self.X_ = X
        self.y_ = y
        
        for i in range(self.epochs):
            self.train(X, y)
        
        return self
    
    def predict_proba(self, X):
        """Calculate the NN inference using feed forward"""
        check_is_fitted(self)
        X = check_array(X)

        # Weighted sum
        hidden_sum = np.dot(X, self.weights_i2h)
        
        # Activation
        activated_hidden = self.sigmoid(hidden_sum)
        
        # Weighted sum 2
        output_sum = np.dot(activated_hidden, self.weights_h2o)
        
        # Final Output
        activated_output = self.sigmoid(output_sum)
        
        return activated_output.reshape(1, -1)

    def predict(self, X):
        check_is_fitted(self)
        pred_proba = self.predict_proba(X)
        return np.round(pred_proba).astype(int)

In [31]:
from sklearn.metrics import accuracy_score

In [42]:
perceptron = Perceptron(epochs=1000).fit(df[['x1', 'x2', 'x3']], df['y'])
y_pred = perceptron.predict(df[['x1', 'x2', 'x3']])
score = accuracy_score(df['y'], y_pred[0])
print(f'Accuracy score: {score * 100:.2f}%')

Accuracy score: 42.86%


In [45]:
perceptron.predict_proba(df[['x1', 'x2', 'x3']])[0]

array([4.98535976e-01, 4.99998954e-01, 4.99998935e-01, 4.98399660e-01,
       4.98123324e-01, 4.99999999e-01, 6.49223463e-05])

In [44]:
y_pred[0]

array([0, 0, 0, 0, 0, 0, 0])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the canonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the Internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [3]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [7]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simplify the problem for now: Zero or all else.

In [9]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [10]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [ ]:
##### Your Code Here #####



## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?